In [25]:
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_ollama.chat_models import ChatOllama
from helpers.prompt import system
from langchain_core.tools import tool
import io
import os
from contextlib import redirect_stdout
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [26]:
mcp = MultiServerMCPClient(
    {
        "github": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/github",
                "--key",
                os.getenv("SMITHERY_KEY"),
                "--profile",
                "superior-beaver-DXJgLy"
            ],
            "transport": "stdio"
        },
        "mcp-atlassian": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "mcp-atlassian",
                "--key",
                os.getenv("SMITHERY_KEY"),
                "--profile",
                "superior-beaver-DXJgLy"
            ],
            "transport": "stdio"
        }
    }
)
mcp_tools = await mcp.get_tools()
llm = ChatOllama(model="qwen3:8b", temperature=0.8)


In [16]:
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

In [17]:
@tool
def python_code_interpreter(code: str, local_vars: dict) -> str:
    """
    Executes a snippet of Python code to perform complex analysis, filtering, and calculations on data.
    The code has access to variables passed on 'local_vars' which is a dictionarie.
    The code MUST print its final result to stdout.
    Example:
    'total = 0
    for pr in prs:
        if pr["state"] == "open":
            total += 1
    print(f"Total open PRs: {total}")'
    """
    output_capture = io.StringIO()

    try:
        with redirect_stdout(output_capture):
            exec(code, {"__builtins__": __builtins__}, local_vars)
        
        result = output_capture.getvalue()
        if not result:
            return "Code executed successfully, but produced no output. Remember to use the print() function."
        return result

    except Exception as e:
        return f"Error executing code: {e}\nCaptured output: {output_capture.getvalue()}"


In [18]:
relevant_tools = ['search_repositories', 'list_pull_requests', 'get_pull_request']
tools = [tool for tool in mcp_tools if tool.name in relevant_tools] + [python_code_interpreter]
llm_tools = llm.bind_tools(tools)

In [19]:
def call_model(state: AgentState):
    return {"messages": [llm_tools.invoke(state['messages'])]}

def reflection_node(state: AgentState):
    last_message = state['messages'][-1]

    human_requests = [msg.content for msg in state["messages"] if isinstance(msg, HumanMessage)]
    original_request = human_requests[0]

    reflection_instruction = system(
        path='prompts/reflection.md',
        last_message = last_message.content,
        original_request = original_request
    )

    return {"messages": [reflection_instruction]}

def check_for_errors(state: AgentState):
    last_message = state['messages'][-1]
    if not isinstance(last_message, ToolMessage):
        return END
    
    if "Error" in last_message.content or "Exception" in last_message.content or "McpError" in last_message.content:
        return "reflection"
    else:
        return "assistant"

    

In [20]:
tool_node = ToolNode(tools)

workflow = StateGraph(AgentState)

workflow.add_node("assistant", call_model)
workflow.add_node("tools", tool_node)
workflow.add_node("reflection", reflection_node)

workflow.add_edge(START, "assistant")
workflow.add_conditional_edges("assistant", tools_condition)
workflow.add_conditional_edges("tools", check_for_errors, {
    "reflection": "reflection",
    "assistant": "assistant"
})
workflow.add_edge("reflection", "assistant")

agent = workflow.compile()

In [22]:
system_prompt = system(path='prompts/system.md', username = 'tenlisboa', companies_usernames = 'px-center')

messages = system_prompt + [HumanMessage(content="How many PRs the user matheuspaz did last month on px-radar-check repo? How would you evaluate his work by that?")]
async for event in agent.astream({"messages": messages}):
    for v in event.values():
        if 'messages' in v:
            v['messages'][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, let's tackle the user's question. They want to know how many PRs matheuspaz opened on the px-radar-check repo in the last month and evaluate their work based on that.

First, I need to figure out how to get the list of pull requests from the px-radar-check repository. The user mentioned the repo name, so I'll use the list_pull_requests function. The owner is px-center since that's the company associated with the GitHub username tenlisboa, but wait, the user is asking about matheuspaz's PRs. Wait, the user's GitHub username is tenlisboa, but the question is about matheuspaz. So the owner here might be px-center, and the repo is px-radar-check.

Wait, the list_pull_requests function requires the owner and repo parameters. The owner is the repository's owner, which in this case is likely px-center since the user's context says their GitHub Companies are px-center. So the owner is px-center, rep